## 1. Install and Import Dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

## 2. Instantiate Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## 3. Encode and Calculate Sentiment

In [3]:
tokens = tokenizer.encode('Mehh, it was okay', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits     #### what position represents the actual sentiment

tensor([[-1.0165,  1.6135,  2.9409, -0.1085, -2.8996]],
       grad_fn=<AddmmBackward>)

In [6]:
int(torch.argmax(result.logits)) + 1 ## That argument shows where the best or maximum sentiment point in our model.    

3

## 4. Collect Reviews

In [7]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

##  5. Load Reviews into DataFrame and Score

In [8]:
import numpy as np
import pandas as pd

In [9]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [10]:
df['review'].iloc[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [11]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [12]:
sentiment_score(df['review'].iloc[3])

5

In [14]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))   ## It's limited to use tokens at the same time because of that we have used 512 here.

In [15]:
df.head()

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,It was ok. Had coffee with my friends. I'm new...,3
2,Great staff and food. Must try is the pan fri...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I came to Social brew cafe for brunch while ex...,5


In [16]:
df['review'].iloc[3]

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"